<a href="https://www.meetup.com/Amazon-Web-Services-Mexico/"><img src="images/AWSCDMX.png"></a>
# MeetUp Grupo de Usuarios AWS CDMX 20190424

# Uso de AWS con Python

## David Jonathan Sol Llaven
T-Systems México

Contacto:
- https://medium.com/@soldavid
- Grupo de Usuarios AWS Ciudad de México: https://www.meetup.com/Amazon-Web-Services-Mexico/
- Canal de Slack: https://launchpass.com/awscdmx

## ¿Cómo manejamos AWS?

### A través de la [Consola AWS](https://aws.amazon.com/console/)

<img src="images/consola.png">

### A través de llamadas API directas

    https://ec2.amazonaws.com/?Action=RunInstances
    &ImageId=ami-2bb65342
    &MaxCount=3
    &MinCount=1
    &Placement.AvailabilityZone=us-east-1a
    &Monitoring.Enabled=true
    &Version=2016-11-15
    &X-Amz-Algorithm=AWS4-HMAC-SHA256
    &X-Amz-Credential=AKIAIOSFODNN7EXAMPLEus-east-1%2Fec2%2Faws4_request
    &X-Amz-Date=20130813T150206Z
    &X-Amz-SignedHeaders=content-type%3Bhost%3Bx-amz-date
    &X-Amz-Signature=ced6826de92d2bdeed8f846f0bf508e8559e98e4b0194b84example54174deb456c
    Content-type: application/json
    host:ec2.amazonaws.com

### A través de [AWS CLI](https://aws.amazon.com/cli/)

    >aws s3 ls  --profile SNDBX
    2018-07-17 15:39:39 amqlkinnsgtw01a-fileshare-smb
    2018-09-20 21:03:54 amsbtinnsgtw01g
    2018-09-06 11:10:39 amseninnaim01g
    2018-11-22 17:58:51 amseninnseg01g
    2018-05-21 15:13:49 cf-templates-fmsc56vfvtdz-us-east-1
    2019-03-26 11:45:38 orquestacion
    2019-02-26 16:04:42 red-hat-openshift-openshiftstack-1-registrybucket-1a1uc2vy8hvhp
    2019-02-26 16:04:43 red-hat-openshift-openshiftstack-1e0-outputbucket-x2ogc6th606q
    2019-02-26 16:04:50 red-hat-openshift-openshiftstack-lambdazipsbucket-s82l02hs48gx

### A través de los [SDKs](https://aws.amazon.com/tools/)

- Javascript
- PHP
- Go
- Ruby
- Java
- Muchos más...

### El SDK de Python se llama [BOTO3](https://aws.amazon.com/sdk-for-python/)

Nos permite el acceso a los recursos de AWS de forma simple, a traves de:

- Clientes: Acceso de Bajo Nivel

      sqs = boto3.client('sqs')

- Recursos: Acceso de Alto Nivel

      s3 = boto3.resource('s3')

Y nos proporciona mecanismos para ejecutar comandos como son:

- Sesiones

      session = boto3.session.Session()
    
- Colecciones

      for bucket in s3.buckets.all():
          print(bucket.name)
    
- Paginadores

      paginator = client.get_paginator('list_objects')
      page_iterator = paginator.paginate(Bucket='my-bucket')
      for page in page_iterator:
          print(page['Contents'])
        
- Waiters

      waiter = client.get_waiter('alarm_exists')

<a href="https://www.python.org/"><img src="images/python-logo.jpg"></a>
## ¿Qué es [Python](https://www.python.org/)?
Python es un lenguaje interpretado de tipos de variables dinámico, que ha ganado mucha popularidad dada su facilidad de aprendizaje, claridad en el código y amplio número de librerias para muchos casos de uso.
* Creado en 1991 por Guido van Rossum
https://www.python.org/

In [23]:
# Importa el SDK para AWS
import boto3
# Abre una sesion con un nombre de perfil definido en el archivo ~/.aws/credentials
session = boto3.session.Session(profile_name='SNDBX')
# Tambiém se puede hacer tuilizando variables de ambiente del sistema operativo:
# AWS_PROFILE
# AWS_ACCESS_KEY_ID
# AWS_SECRET_ACCESS_KEY
# Para mas información ver https://boto3.amazonaws.com/v1/documentation/api/latest/guide/configuration.html
# Usando esa sesión se abre una conexión tipo recurso a S3
s3 = session.resource('s3')
# Se obtienen todos los buckets en esa cuenta, y se imprime el nombre de cada una de ellas
for bucket in s3.buckets.all():
    print(bucket.name)

amqlkinnsgtw01a-fileshare-smb
amsbtinnsgtw01g
amseninnaim01g
amseninnseg01g
cf-templates-fmsc56vfvtdz-us-east-1
orquestacion
red-hat-openshift-openshiftstack-1-registrybucket-1a1uc2vy8hvhp
red-hat-openshift-openshiftstack-1e0-outputbucket-x2ogc6th606q
red-hat-openshift-openshiftstack-lambdazipsbucket-s82l02hs48gx


In [38]:
bucket = s3.Bucket('orquestacion')
fecha = bucket.creation_date.strftime('%Y-%m-%d a las %H:%M')
print(f'Bucket {bucket.name} creada el {fecha}')

Bucket orquestacion creada el 2019-03-26 a las 17:45


In [42]:
import pprint
s3client = session.client('s3')
paginator = s3client.get_paginator('list_objects')
page_iterator = paginator.paginate(Bucket='orquestacion')
for page in page_iterator:
          pprint.pprint(page['Contents'])

[{'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
  'Key': 'documentos/',
  'LastModified': datetime.datetime(2019, 3, 26, 17, 57, 27, tzinfo=tzutc()),
  'Owner': {'DisplayName': 'sndbx.senha',
            'ID': 'ac38b603a5d171013630d2f8adde4648f78926ea3f5e34f0fd45d904e32d876f'},
  'Size': 0,
  'StorageClass': 'STANDARD'},
 {'ETag': '"24802cc18703059034862d8748428392-2"',
  'Key': 'documentos/02_02 Portal único/01 Memorias Técnicas/01 SENHA AGS M T '
         'Red Hat Ansible Tower_firmado.pdf',
  'LastModified': datetime.datetime(2019, 3, 27, 2, 52, 1, tzinfo=tzutc()),
  'Owner': {'DisplayName': 'sndbx.senha',
            'ID': 'ac38b603a5d171013630d2f8adde4648f78926ea3f5e34f0fd45d904e32d876f'},
  'Size': 13990702,
  'StorageClass': 'STANDARD'},
 {'ETag': '"09b0af2cd7459510fd8d63c1dfc0cf2e"',
  'Key': 'documentos/02_02 Portal único/01 Memorias Técnicas/01 SENHA AGS M T '
         'Red Hat Ansible Tower_versión final.docx',
  'LastModified': datetime.datetime(2019, 3, 27, 2, 52, 15, tzin

  'LastModified': datetime.datetime(2019, 3, 28, 6, 54, 13, tzinfo=tzutc()),
  'Owner': {'DisplayName': 'sndbx.senha',
            'ID': 'ac38b603a5d171013630d2f8adde4648f78926ea3f5e34f0fd45d904e32d876f'},
  'Size': 13004071,
  'StorageClass': 'STANDARD'},
 {'ETag': '"0742a137ec63ba5474f2983c0b3d43ae"',
  'Key': 'entrega_sat/01_Nube_Privada/08 Información de Soporte a la '
         'operación/Organigrama Operación SENHA_versión final.pptx',
  'LastModified': datetime.datetime(2019, 3, 28, 6, 54, 13, tzinfo=tzutc()),
  'Owner': {'DisplayName': 'sndbx.senha',
            'ID': 'ac38b603a5d171013630d2f8adde4648f78926ea3f5e34f0fd45d904e32d876f'},
  'Size': 2057245,
  'StorageClass': 'STANDARD'},
 {'ETag': '"8ce89c0f913ee73e38e7b5198e65590c"',
  'Key': 'entrega_sat/01_Nube_Privada/08 Información de Soporte a la '
         'operación/Plantilla Incidentes Críticos_versión final.xlsb',
  'LastModified': datetime.datetime(2019, 3, 28, 6, 54, 18, tzinfo=tzutc()),
  'Owner': {'DisplayName': 'sndb

  'Owner': {'DisplayName': 'sndbx.senha',
            'ID': 'ac38b603a5d171013630d2f8adde4648f78926ea3f5e34f0fd45d904e32d876f'},
  'Size': 3290213,
  'StorageClass': 'STANDARD'},
 {'ETag': '"af621cd4c0eae1450d0a1c1ceefaf4bc"',
  'Key': 'entrega_sat/02_PANH/02_01 Orquestador/8 SARI/Anexo A/SAT-SENHA CUs '
         'SARI-a.6 Asignación de NFS existente a una o más máquinas virtuales '
         'FIRMADO.pdf',
  'LastModified': datetime.datetime(2019, 3, 28, 7, 2, 10, tzinfo=tzutc()),
  'Owner': {'DisplayName': 'sndbx.senha',
            'ID': 'ac38b603a5d171013630d2f8adde4648f78926ea3f5e34f0fd45d904e32d876f'},
  'Size': 3269075,
  'StorageClass': 'STANDARD'},
 {'ETag': '"bf6f78a460d1836dabec45eee4275ba1"',
  'Key': 'entrega_sat/02_PANH/02_01 Orquestador/8 SARI/Anexo A/SAT-SENHA CUs '
         'SARI-a.6 Asignación de NFS existente a una o más máquinas '
         'virtuales.docx',
  'LastModified': datetime.datetime(2019, 3, 28, 7, 2, 15, tzinfo=tzutc()),
  'Owner': {'DisplayName': 'sndbx.s

         'AGS M T Red Hat Cloudforms_firmado.pdf',
  'LastModified': datetime.datetime(2019, 3, 28, 7, 47, 44, tzinfo=tzutc()),
  'Owner': {'DisplayName': 'sndbx.senha',
            'ID': 'ac38b603a5d171013630d2f8adde4648f78926ea3f5e34f0fd45d904e32d876f'},
  'Size': 17161571,
  'StorageClass': 'STANDARD'},
 {'ETag': '"806c2e8559392a9a58f73244f2693e89"',
  'Key': 'entrega_sat/02_PANH/02_02 Portal único/01 Memorias Técnicas/02 SENHA '
         'AGS M T Red Hat Cloudforms_versión final.docx',
  'LastModified': datetime.datetime(2019, 3, 28, 7, 50, 44, tzinfo=tzutc()),
  'Owner': {'DisplayName': 'sndbx.senha',
            'ID': 'ac38b603a5d171013630d2f8adde4648f78926ea3f5e34f0fd45d904e32d876f'},
  'Size': 1195396,
  'StorageClass': 'STANDARD'},
 {'ETag': '"8ee86df6725ec44521b87fad821cf0e6-3"',
  'Key': 'entrega_sat/02_PANH/02_02 Portal único/01 Memorias Técnicas/02 SENHA '
         'QRO M T Red Hat CloudForms_FIRMADO.pdf',
  'LastModified': datetime.datetime(2019, 3, 28, 7, 50, 58, tzinfo=

 {'ETag': '"da9a95722cb449113a50e67b5c547434"',
  'Key': 'entrega_sat/02_PANH/02_03 Servicios de Soporte a la Operación/03 '
         'Procesos SENHA/Gestión de Conocimiento/Anexos/2.6 Bitácora '
         'Operativa/Reporte Seguimiento Diario.xlsx',
  'LastModified': datetime.datetime(2019, 3, 28, 8, 26, 11, tzinfo=tzutc()),
  'Owner': {'DisplayName': 'sndbx.senha',
            'ID': 'ac38b603a5d171013630d2f8adde4648f78926ea3f5e34f0fd45d904e32d876f'},
  'Size': 1609732,
  'StorageClass': 'STANDARD'},
 {'ETag': '"b0a8be5fb9d8316cfd2c4831fe30da7d"',
  'Key': 'entrega_sat/02_PANH/02_03 Servicios de Soporte a la Operación/03 '
         'Procesos SENHA/Gestión de Conocimiento/Anexos/2.7 Reporte de RFCs '
         'que modificaron CMDB/10.Reporte NServicio '
         'CAMBIOS_OCTUBRE2018_SENHA.xls',
  'LastModified': datetime.datetime(2019, 3, 28, 8, 26, 21, tzinfo=tzutc()),
  'Owner': {'DisplayName': 'sndbx.senha',
            'ID': 'ac38b603a5d171013630d2f8adde4648f78926ea3f5e34f0fd45d904e